# Классификация тональности комментариев

Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Предоставлен набор данных с разметкой о токсичности правок.


### Цель

Построить модель, способную классифицировать комментарии на позитивные и негативные.

### Задачи

* Загрузить и подготовить данные
* Обучить разные модели
* Выбрать лучшую модель
* Протестировать выбранную модель

### Описание данных

Имеется набор данных с разметкой о токсичности правок.


Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем все необходимые инструменты

In [1]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import (train_test_split, GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.utils import shuffle
from sklearn.metrics import (
    f1_score, classification_report)

from tqdm import tqdm

# настройки
import warnings
warnings.filterwarnings("ignore")

Загрузим данные 

In [2]:
df = pd.read_csv('toxic_comments.csv', index_col=0)

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Для ускорения вычислений оставим только 10 % исходной выборки.

In [5]:
df = df.sample(frac=0.1, random_state=12345)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15929 entries, 109583 to 75311
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15929 non-null  object
 1   toxic   15929 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 373.3+ KB


**Обработаем исходные данные**

In [7]:
df.reset_index(inplace=True)

In [8]:
df['toxic'] = df['toxic'].astype('bool')

In [9]:
df.duplicated().sum()

0

Очистим тексты от лишних символов и стоп-слов

In [10]:
# Функция для очистки текстов от лишних символов 
def clear_symbols(text):
    clear_text = re.sub(r'[^A-Za-z]+', ' ', text).lower() 
    return ' '.join(clear_text.split())

In [11]:
# Функция для очистки текстов от стоп-слов
def clear_stop_words(text, stopwords):
    clear_text = [word for word in text.split() 
                  if word not in stopwords]
    return ' '.join(clear_text)

In [12]:
# Загрузим стоп-слова
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
np.array(stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Степан\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array({'ll', 'they', 'my', 'ours', 'there', 'each', "mightn't", 'itself', 's', "needn't", 'you', 'i', 've', 'he', "you've", 'him', 'this', 'y', 'shouldn', 'hadn', 'she', 'which', 'been', 'hasn', 'same', "wasn't", 'is', 'has', "that'll", "you'll", 'down', 'ain', 'her', 'his', 'such', "wouldn't", 'them', 'themselves', 'our', 'did', 'a', 'more', 'off', 'wasn', "it's", 'himself', 'over', 'until', 'who', 't', 'wouldn', 'will', 'those', 'nor', 'above', "mustn't", 'had', 'then', 'now', 'o', 'for', "didn't", 'of', 'after', 'isn', "you're", 'on', 'have', 'again', 'once', 'd', 'all', "haven't", 'ourselves', 'because', "aren't", "she's", 'having', 'own', 'at', 'under', 'doing', 'didn', "isn't", 'whom', 'what', 'these', 'out', 'can', 'where', 'weren', "weren't", 'it', 'hers', 'very', 'theirs', 'below', 'against', 'only', 'doesn', 'while', 'about', "hadn't", 'do', 'yourselves', 'into', 'was', 'not', 'and', 'than', 'why', 'with', 'before', 'few', 'no', 'mustn', 'shan', 'to', 'herself', "shan't", 'th

In [13]:
df['clear_text'] = df['text'].apply(
    lambda x: clear_symbols(
    clear_stop_words(str(x), stopwords)))

In [14]:
df.isna().sum()

index         0
text          0
toxic         0
clear_text    0
dtype: int64

**Лемматизируем тексты**

In [15]:
def lemmatize(df, text_column, n_samples):
    """ 
    n_samples - количество текстов для объединения.
    Лемматизация объединенных спец-символом "br" текстов и их последующее 
    разбиение по этому символу позволяют съэкономить время.
    """
    result = []

    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

    for i in tqdm(range((df.shape[0] // n_samples) + 1)):
        start = i * n_samples
        stop = start + n_samples

        sample = '?'.join(
            df[text_column][start : stop].values)
        lemmas = nlp(sample)
        # lemm_sample = ''.join(lemmas).split('?')
        lemm_sample = " ".join([token.lemma_ for token in lemmas]).split('?')

        result += lemm_sample

    return pd.Series(result, index=df.index)

In [16]:
df['lemmatized_text'] = lemmatize(
    df=df,
    text_column='clear_text',
    n_samples=3000)

100%|██████████| 6/6 [01:06<00:00, 11.10s/it]


In [19]:
df.head()

,text,clear_text,lemmatized_text,toxic
0,Expert Categorizers \n\nWhy is there no menti...,expert categorizers why mention fact nazis par...,expert categorizer why mention fact nazi parti...,False
1,"""\n\n Noise \n\nfart* talk. """,noise fart talk,noise fart talk,True
2,"An indefinite block is appropriate, even for a...",an indefinite block appropriate even minor inf...,an indefinite block appropriate even minor inf...,False
3,I don't understand why we have a screenshot of...,i understand screenshot ap s gui ub can someon...,i understand screenshot ap s gui ub can someon...,False
4,"Hello! Some of the people, places or things yo...",hello some people places things written articl...,hello some people place thing write article ni...,False


In [20]:
df = df[['text', 'clear_text', 'lemmatized_text', 'toxic']]

**Разделим выборку на тестовую, валидационную и обучающую. Выделим features и target.**

In [21]:
train, test = train_test_split(df, test_size=0.1, random_state=12345)

In [22]:
features_train = train['lemmatized_text']
features_test = test['lemmatized_text']

target_train = train['toxic']
target_test = test['toxic']

In [23]:
# сравним распределение целевого признака
for sample in [train, test]:
    print(sample[sample['toxic']==1].shape[0]/sample.shape[0])

0.10142299107142858
0.09918392969240426


Имеем примерно 10% негативных комментариев в каждой выборке. Будем учитывать дисбаланс методом взвешивания классов.

**Получим TF-IDF векторы**

In [24]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [25]:
features_train = vectorizer.fit_transform(features_train)
features_test = vectorizer.transform(features_test)

In [27]:
display(features_train.shape)
display(features_test.shape)

(14336, 36991)

(1593, 36991)

Итак, теперь для каждого текста мы имеем векторное представление. 

## Обучение

**DecisionTreeClassifier**

In [29]:
tree = DecisionTreeClassifier(random_state=12345, class_weight='balanced')

In [30]:
GS_tree = GridSearchCV(estimator = tree, 
                  param_grid = {'max_depth':[x for x in range(5, 30, 5)]},
                  scoring = ["f1"], 
                  refit = "f1",
                  cv=3
                  )

In [35]:
GS_tree.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=12345),
             param_grid={'max_depth': [5, 10, 15, 20, 25]}, refit='f1',
             scoring=['f1'])

In [38]:
best_tree = GS_tree.best_estimator_
GS_tree.best_score_.round(2)

0.55

f1-мера для DecisionTreeClassifier равна 0.55

**LogisticRegression**

In [39]:
logistic = LogisticRegression(random_state=12345, class_weight='balanced')

In [40]:
GS_logistic = GridSearchCV(estimator = logistic,
                  param_grid = {'C':[9, 10, 11],
                                'solver':['lbfgs','sag'],
                                'max_iter':[14, 16, 18]},
                  scoring = ["recall", "precision", "f1"], #sklearn.metrics.SCORERS.keys()
                  refit = "f1",
                  cv=3
                  )

In [41]:
GS_logistic.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=LogisticRegression(class_weight='balanced',
                                          random_state=12345),
             param_grid={'C': [9, 10, 11], 'max_iter': [14, 16, 18],
                         'solver': ['lbfgs', 'sag']},
             refit='f1', scoring=['recall', 'precision', 'f1'])

In [43]:
best_logistic = GS_logistic.best_estimator_
GS_logistic.best_score_.round(2)

0.72

f1-мера для LogisticRegression равна 0.72

**RandomForestClassifier**

In [45]:
forest = RandomForestClassifier(random_state=12345, class_weight='balanced')

In [46]:
GS_forest = GridSearchCV(estimator = forest,
                  param_grid = {"n_estimators" : 
                                [x for x in range(8, 13, 2)]},
                  scoring = ["recall", "precision", "f1"],
                  refit = "f1",
                  cv=3
                  )

In [47]:
GS_forest.fit(features_train, target_train)

GridSearchCV(cv=3,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=12345),
             param_grid={'n_estimators': [8, 10, 12]}, refit='f1',
             scoring=['recall', 'precision', 'f1'])

In [49]:
best_forest = GS_forest.best_estimator_
GS_forest.best_score_.round(2)

0.4

f1-мера для RandomForestClassifier равна 0.4.

**Создадим константную модель**

In [50]:
dummy_clf = DummyClassifier(strategy="constant", constant=1)
dummy_clf.fit(features_train, target_train)
dummy_predictions = dummy_clf.predict(features_test)
f1_forest = f1_score(target_test, dummy_predictions).round(2)
print(classification_report(target_test, dummy_predictions, target_names=['Не токсичный', 'Токсичный']))
print('f1: ', f1_forest)


              precision    recall  f1-score   support

Не токсичный       0.00      0.00      0.00      1435
   Токсичный       0.10      1.00      0.18       158

    accuracy                           0.10      1593
   macro avg       0.05      0.50      0.09      1593
weighted avg       0.01      0.10      0.02      1593

f1:  0.18


**Протестируем лучшую модель**

In [52]:
predictions_test = best_logistic.predict(features_test)

In [54]:
f1_score(target_test, predictions_test).round(2)

0.76

Требуемое значение f1-меры было получено.

## Выводы

Целью данного проекта было построение модели, способной классифицировать комментарии на позитивные и негатиные. 

В качестве исходных данных был получен корпус из 159 тысяч текстов, около 10% из которых были тосичными. 

Тексты были очищены от лишних слов и символов, слова в них были лемматизированы. Также был учтен дисбаланс классов методом upsampling.
Затем тексты были векторизированы с помощью TF-IDF.

Для решения задачи классификации были выбраны три модели:
* DecisionTreeClassifier;
* LogisticRegression;
* RandomForestClassifier.

С учетом подобранных гиперпараметров наилучший результат показала модель логистической регрессии. Для нее также был подобран порог классификации.

На тестовой выборке для выбранной модели значение f1-меры составило 
0.76. Так как для ускорения вычислений в работе использовались только 10% от всех данных, при использовании всей выборки точность предсказаний должна возрасти.